In [11]:
from PIL import Image
import numpy as np

# Load the image
img = Image.open("part2-image1.jpg")

# Convert the image to a numpy array
img_array = np.array(img)

# Flatten each row of the image
images = img_array.reshape(img_array.shape[0], -1, 3)

# Now, flattened_img_array is a 3D array where each row is a flattened version of the corresponding row in the original image

In [17]:
# Assume images is a 3D array with shape (n_images, n_pixels, 3)
# Each row is a flattened image and the last dimension are the RGB channels
import networkx as nx
compressed_images = []
n_components = 2

for channel in range(3):
    # Extract the current color channel
    images_channel = images[:, :, channel]

    # Compute the Euclidean distance matrix
    dist_matrix = np.sqrt(
        np.sum((images_channel[:, None] - images_channel) ** 2, axis=-1)
    )

    # Construct a k-nearest neighbors graph
    neighbors = np.argsort(dist_matrix, axis=1)[:, 1 : k + 1]
    graph = nx.from_numpy_array(dist_matrix)
    graph.remove_edges_from(graph.edges)
    for i in range(images_channel.shape[0]):
        for j in neighbors[i]:
            graph.add_edge(i, j, weight=dist_matrix[i, j])

    # Compute the shortest path distance matrix
    shortest_path_dist_matrix = np.array(nx.floyd_warshall_numpy(graph))

    # Apply classical MDS
    H = (
        np.eye(images_channel.shape[0])
        - np.ones((images_channel.shape[0], images_channel.shape[0]))
        / images_channel.shape[0]
    )
    B = -H @ (shortest_path_dist_matrix**2) @ H / 2
    eigvals, eigvecs = np.linalg.eigh(B)

    # The lower-dimensional embedding is given by the top k eigenvectors scaled by the square root of the corresponding eigenvalues
    idx = np.argsort(eigvals)[::-1][:n_components]
    compressed_images_channel = eigvecs[:, idx] @ np.diag(np.sqrt(eigvals[idx]))

    compressed_images.append(compressed_images_channel)

# Stack the compressed images for each channel along the last dimension
compressed_images = np.stack(compressed_images, axis=-1)

In [18]:
import cv2
cv2.imwrite("isomap.jpg", compressed_images)

True

In [19]:
def knn_predict(X_train, y_train, X_test, k):
    # Compute the Euclidean distance between each test point and all training points
    dists = np.sqrt(np.sum((X_test[:, None] - X_train) ** 2, axis=-1))

    # Find the k nearest neighbors
    neighbors = np.argsort(dists, axis=1)[:, :k]

    # Compute the mean of the neighbors
    return np.mean(y_train[neighbors], axis=1)


# Use the KNN regressor to decompress the images for each color channel
decompressed_images = []
for channel in range(3):
    decompressed_images_channel = knn_predict(
        compressed_images[:, :, channel],
        images[:, :, channel],
        compressed_images[:, :, channel],
        k=5,
    )
    decompressed_images.append(decompressed_images_channel)

# Stack the decompressed images for each channel along the last dimension
decompressed_images = np.stack(decompressed_images, axis=-1)

# Reshape the first decompressed image to the original image shape
image_height, image_width = (
    1280,
    900,
)  # replace with the height and width of your original image
decompressed_image = decompressed_images[0].reshape(image_height, image_width, 3)

# Save the decompressed image
decompressed_image = Image.fromarray((decompressed_image * 255).astype(np.uint8))
decompressed_image.save("decompressed_image.jpg")

ValueError: cannot reshape array of size 2700 into shape (1280,900,3)